# Introduction: Graph Setup

We will use this notebook to explore `py2neo` and how to connect to our linked Neo4J graph. Once we are able to make a connection to the graph, we can query on it, and use our favorite Python data science libraries to make further insights.

In [1]:
from py2neo import Graph

### Neo4J Connection

We connect to a linked neo4j docker instance within our bridged network. Since this is a local demonstration, security is not a concern.

In [2]:
uri = 'http://neo4j:7474/'
graph = Graph(uri, user='neo4j', password='test', name='cgraph')

ConnectionUnavailable: ('Cannot open connection to %r', ConnectionProfile('http://neo4j@neo4j:7474'))

### Neosemantics Configuration

Calling Neosemantics `graphconfig.init()` to load the constraints for dealing with Linked Data.

In [3]:
# If graph isn't empty, don't initialize graphconfigs
if graph.match_one() == None:
    graph.call.n10s.graphconfig.init()
graph.call.n10s.graphconfig.show()

(No data)

In the Omniscience taxonomy, there are prefixes available to make the graph readable, and forces `Neosemantics` to use appropriate namespace prefixes.

In [5]:
graph.call.apoc.cypher.runFile('/import/taxonomy/omniscience/setup/namespaces.cypher')

row,result
0,"{prefix: 'a1', namespace: 'http://www.elsevier.com/xml/schema/grant/grant-1.1'}"
1,"{prefix: 'a2', namespace: 'http://www.elsevier.com/xml/schema/grant/grant-1.2'}"
2,"{prefix: 'arg', namespace: 'http://spinrdf.org/arg#'}"


Creating a uniqueness constraint on Resources. Resources are defined as nodes that can be dereferenced via their `uri` property.

In [9]:
if not graph.schema.get_uniqueness_constraints('Resource'):
    graph.schema.create_uniqueness_constraint('Resource', 'uri')

### Data Loading

We will use `Neosemantics` to load a taxonomy in the existing bibliometric C-Graph database.

#### Loading OmniScience

In [8]:
graph.run("CALL n10s.rdf.import.fetch('file:///import/omniscience/statements.ttl', 'Turtle')")

terminationStatus,triplesLoaded,triplesParsed,namespaces,extraInfo,callParams
KO,0,0,null,A Graph Config is required for RDF importing procedures to run,null


@TODO: Some experiment calculating hIndex on a concept.

In [ ]:
def hIndex(citation_counts: list):
    citation_counts.sort(reverse=True)
    for index, citation_count in enumerate(citation_counts):
        if index > citation_count:
            return index
    return len(citation_counts)